In [2]:
!pip3 install 'crewai[tools]' convertapi anthropic langchain-anthropic > /dev/null

In [1]:
from crewai import Agent,Task,Crew
from crewai_tools import BaseTool
from dotenv import load_dotenv
import convertapi
import os
from langchain_anthropic import ChatAnthropic
load_dotenv(".env")


True

In [2]:
convertapi.api_secret = os.environ["CONVERT_API_KEY"]
Haiku = ChatAnthropic(
    model_name="claude-3-haiku-20240307"
)

In [3]:
class Splitter(BaseTool):
    name: str = "Splitter"
    description: str = "Used by agents to split pdf by pages and store them in a folder.'pdf_path' and 'output_directory' will be given as arguments."

    def _run(self,pdf_path,output_directory):
         convertapi.convert('split',{
              'File':pdf_path
         },from_format='pdf').save_files(output_directory)

class Maker(BaseTool):
    name: str = "Maker"
    description: str = "Used by agents to create a new fresh directory.It will be used further in other tasks.Returns the name of the created directory.'directory_name' will be given as arguments"

    def _run(self,directory_name):
        os.mkdir(directory_name)
        return directory_name

class WaterMarker(BaseTool):
     name: str = "WaterMarker"
     description: str = "Used by agents to watermark all the pdfs in the given directory.'directory' will be given as argument"

     def _run(self,directory):
        files = os.listdir(directory)
        for file in files:
            convertapi.convert('watermark', {
    'File': directory+"/"+file,
    'Text': 'MLX'
}, from_format = 'pdf').save_files(directory)

In [4]:
Handler =  Agent(
    role="Pdf Master",
    goal="Performing various options on pdf by selecting appropriate tools.",
    backstory="""You are an expert at working with pdfs.
    You  are assigned to Extensively work with pdf.
    You are capable of taking decisions on which tool to use from your diverse set of tools""",
    verbose=True,
    allow_delegation=False,
    llm=Haiku,
    tools=[Splitter(),WaterMarker(),Maker()],
)

In [5]:

task1 = Task(
    description="On given pdf named 'Safari.pdf',Please split the pdfs into seperate pages in a fresh directory called 'crew-results'",
    expected_output="Splitted pdf pages on the newly created directory ",
    agent=Handler
)

task2 = Task(
    description="On the created directory where the splitted pdfs are stored, Watermark all the pdfs",
    expected_output="Watermarked pdfs in the created directory",
    agent=Handler
)

In [6]:
crew = Crew(
    agents=[Handler],
    tasks=[task1,task2,],
    verbose=2
)

In [7]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: Pdf Master
 [INFO]: == Starting Task: You have to create a new fresh directory which can be used in other tasks. The directory name should be 'crew-results'


> Entering new CrewAgentExecutor chain...
Thought: To create a new fresh directory, I will use the Maker() tool.
Action: Maker
Action Input: {"directory_name": "crew-results"} 

crew-results

Thought: I now know the final answer.
Final Answer: crew-results

> Finished chain.
 [DEBUG]: == [Pdf Master] Task output: crew-results


 [DEBUG]: == Working Agent: Pdf Master
 [INFO]: == Starting Task: On given pdf named 'Safari.pdf',Please split the pdfs into seperate pages in directory created from previous task.


> Entering new CrewAgentExecutor chain...
Thought: To split the 'Safari.pdf' file into separate pages, I will first need to create a directory to store the split pages. I will use the Maker() tool to create a new directory.

Action: Maker
Action Input: {"directory_name": "safari_pages"}
 

safari_pa

KeyboardInterrupt: 